In [1]:
%%time

# 加载llm和embeddings
%run ../utils2.py

from llama_index.core import Settings

# Settings.llm=get_llm('ERNIE-Speed-8K') # 执行建索引时报错 TypeError: 'NoneType' object is not subscriptable
# Settings.llm=get_llm('ERNIE-4.0-8K') # 能正常回答，但也是将不相关的上下文写入答案
# Settings.llm=get_llm('ERNIE-3.5-8K-0205') # 执行建索引时报错 TypeError: 'NoneType' object is not subscriptable
# Settings.llm=get_llm('qwen-max')  # 能正常回答，但也是将不相关的上下文写入答案
# Settings.llm=get_llm('qwen-turbo') # 和qwen2-7b问题类似，将不相关的上下文写入答案
# Settings.llm=get_llm('gpt-4-turbo') # 好用！钱下去的也快～
# Settings.llm=get_llm('gpt-3.5-turbo') # 回答不正确，重复了问题，而且回答是英文
# Settings.llm=get_llm('glm-4') # 回答内容正确，但怎么是英文？
Settings.llm=get_llm('qwen2-7b-6k') # 回答不太正确，将不相关的上下文写入了答案
Settings.embed_model=get_embedding()

CPU times: user 3.41 s, sys: 371 ms, total: 3.78 s
Wall time: 3.43 s


In [2]:
%%time

import requests
from gne import GeneralNewsExtractor

def get_news_data(url):
    response = requests.get(url)
    html = response.text

    extractor = GeneralNewsExtractor()
    result = extractor.extract(html, noise_node_list=[
                               '//div[@class="comment-list"]'])
    result['url']=url
    return result

CPU times: user 9.48 ms, sys: 165 µs, total: 9.64 ms
Wall time: 9.18 ms


In [3]:
%%time

!rm -rf ./index

CPU times: user 3.7 ms, sys: 7.95 ms, total: 11.7 ms
Wall time: 121 ms


In [4]:
%%time

import os
from llama_index.core import(
    StorageContext, 
    load_index_from_storage,
    get_response_synthesizer,
    DocumentSummaryIndex,
    Document
)
from llama_index.core.node_parser import SentenceSplitter

def _load_index(folder_path):
    # 考虑不存在的情况，创建空的索引
    if not os.path.exists(folder_path):
        splitter = SentenceSplitter(chunk_size=1024)
        response_synthesizer = get_response_synthesizer(
            response_mode="tree_summarize", use_async=True,
        )
        
        doc_summary_index = DocumentSummaryIndex.from_documents(
            [],
            transformations=[splitter],
            response_synthesizer=response_synthesizer,
            show_progress=True,
        )
        
        doc_summary_index.storage_context.persist(folder_path)
        
    storage_context = StorageContext.from_defaults(persist_dir=folder_path)
    return load_index_from_storage(storage_context)
    

CPU times: user 0 ns, sys: 24 µs, total: 24 µs
Wall time: 26 µs


In [5]:
%%time

def _data2doc(news_data):
    document=Document(text=news_data['content'], 
                  metadata={"title": news_data['title'],
                            'publish_time': news_data['publish_time'],
                            'author': news_data['author'],
                            'url': news_data['url'],
                            'images': news_data['images'],
                           })
    document.doc_id = document.metadata["title"]
    return document

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 3.58 µs


In [6]:
%%time

doc_summary_index=_load_index('index')

news_data=get_news_data("https://www.guancha.cn/internation/2024_06_20_738635.shtml")
document=_data2doc(news_data)
doc_summary_index.insert(document)
doc_summary_index.refresh([document])
doc_summary_index.storage_context.persist('index')

Parsing nodes: 0it [00:00, ?it/s]

Summarizing documents: 0it [00:00, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

current doc id: 沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”
CPU times: user 472 ms, sys: 32.1 ms, total: 504 ms
Wall time: 12.9 s


In [9]:
%%time

doc_summary_index=_load_index('index')

query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", 
    streaming=True,
    # similarity_top_k=5
)

response = query_engine.query("美国对印度的全球策略态度在过去是如何变化的？如果上下文没有相关知识，则回答不知道，也不要提及上下文知识。")
response.print_response_stream()

不知道。上述提供的信息主要关注沙特麦加酷热期间朝觐者死亡的情况，并未涉及美国对印度的全球策略态度的历史变化。CPU times: user 61.8 ms, sys: 2.95 ms, total: 64.7 ms
Wall time: 2.86 s


In [10]:
query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", 
    streaming=True,
    # similarity_top_k=5
)

response = query_engine.query("沙特麦加发生了什么？如果上下文没有相关知识，则回答不知道，也不要提及上下文知识。")
response.print_response_stream()

沙特麦加近期发生了一起因酷热天气导致的朝觐者死亡事件。每年六月是沙特最热的月份之一，也是朝觐季节，吸引了来自世界各地约180万穆斯林参加为期数天的活动。其中大约160万人来自国外，包括一些年老体弱的人。由于经济原因和签证问题，许多低收入国家的朝觐者没有接受过充分的医疗保健，并且试图在未获得官方许可的情况下参加朝圣。

极端高温导致了大量中暑病例，沙特官方报告称上周日（16日）有2700多例中暑病例。尽管缺乏具体数据，但据法新社统计，截至当地时间6月19日，已有超过900名朝觐者死亡，其中仅埃及人的死亡人数就从18日的300多人跃升至至少600人。一名阿拉伯地区的外交官表示，大部分死者死于高温。

沙特官方称今年的朝圣活动“成功”，并强调了先进的冷却系统和持续供应水源确保了朝觐顺利、安全进行。然而，许多朝觐仪式在白天举行，朝觐者需要长时间户外活动，导致有人中暑晕倒，并且有尸体停放路边。志愿者向朝觐者分发冷饮和冰淇淋以缓解高温影响。

德国气候分析研究所的科学顾问卡尔-弗里德里希·施洛伊斯纳指出，随着全球变暖，未来数千万朝圣者将面临更频繁的高温致死事件。近期一项研究显示，到2030年代全球气温可能升高1.5摄氏度，这将使朝圣者中暑的风险增加五倍。

麦加医疗中心在19日宣布了死亡名单上的死者姓名和国籍，并要求只有死者亲属才能进入大楼确认身份。

In [15]:
query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", 
    streaming=True,
    similarity_top_k=1
)

response = query_engine.query("我国海警怎样处理菲律宾方面的挑衅行动？如果上下文没有相关知识，则回答不知道，也不要提及上下文知识。")
response.print_response_stream()

根据提供的信息，无法得知我国海警如何具体处理菲律宾方面的挑衅行动。这些信息主要关注了沙特麦加朝圣期间的高温问题以及由此导致的朝觐者死亡事件，并未涉及中国与菲律宾之间的海上或任何相关冲突情况。因此，关于这个问题的具体答案是未知的。

In [16]:
len(response.source_nodes)

4

In [18]:
response.source_nodes[0].metadata

{'title': '沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”',
 'publish_time': '2024-06-20 15:24:32',
 'author': '齐倩',
 'url': 'https://www.guancha.cn/internation/2024_06_20_738635.shtml',
 'images': ['https://i.guancha.cn/news/vcg/2024/06/20/20240620143519414.jpg',
  'https://i.guancha.cn/news/vcg/2024/06/20/20240620143539769.jpg']}

In [22]:
doc_summary_index.ref_doc_info

{'沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”': RefDocInfo(node_ids=['63460949-8978-4d84-b00a-8b141e5018ce', 'd7baa6c9-f455-43b4-906b-5cea63642689', '4ec04f1b-06d1-461d-8006-36c8bafa00dc', 'd6fdd7bb-454d-4105-af21-c8a9fc48d8b2', '0871a648-727c-428c-a12a-34ef65c6d3e6'], metadata={'title': '沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”', 'publish_time': '2024-06-20 15:24:32', 'author': '齐倩', 'url': 'https://www.guancha.cn/internation/2024_06_20_738635.shtml', 'images': ['https://i.guancha.cn/news/vcg/2024/06/20/20240620143519414.jpg', 'https://i.guancha.cn/news/vcg/2024/06/20/20240620143539769.jpg']})}

In [27]:
doc_info=doc_summary_index.ref_doc_info['沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”']
doc_info

RefDocInfo(node_ids=['63460949-8978-4d84-b00a-8b141e5018ce', 'd7baa6c9-f455-43b4-906b-5cea63642689', '4ec04f1b-06d1-461d-8006-36c8bafa00dc', 'd6fdd7bb-454d-4105-af21-c8a9fc48d8b2', '0871a648-727c-428c-a12a-34ef65c6d3e6'], metadata={'title': '沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”', 'publish_time': '2024-06-20 15:24:32', 'author': '齐倩', 'url': 'https://www.guancha.cn/internation/2024_06_20_738635.shtml', 'images': ['https://i.guancha.cn/news/vcg/2024/06/20/20240620143519414.jpg', 'https://i.guancha.cn/news/vcg/2024/06/20/20240620143539769.jpg']})

In [31]:

doc_summary_index.docstore.get_node('63460949-8978-4d84-b00a-8b141e5018ce')

TextNode(id_='63460949-8978-4d84-b00a-8b141e5018ce', embedding=None, metadata={'title': '沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”', 'publish_time': '2024-06-20 15:24:32', 'author': '齐倩', 'url': 'https://www.guancha.cn/internation/2024_06_20_738635.shtml', 'images': ['https://i.guancha.cn/news/vcg/2024/06/20/20240620143519414.jpg', 'https://i.guancha.cn/news/vcg/2024/06/20/20240620143539769.jpg']}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'title': '沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”', 'publish_time': '2024-06-20 15:24:32', 'author': '齐倩', 'url': 'https://www.guancha.cn/internation/2024_06_20_738635.shtml', 'images': ['https://i.guancha.cn/news/vcg/2024/06/20/20240620143519414.jpg', 'https://i.guancha.cn/news/vcg/2024/06/20/20240620143539769.jpg']}, hash='9978d66b0a0b205b00a0d0fd2d80ade5f58f0fc0ef34a76147a97d9b8ebe057e'), 